In [1]:
import os
os.chdir('/home/potusvn/Projects/kickstarter_prediction')

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Input, Dense, Dropout
from keras import models, optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from os.path import join
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer
from settings import *

/home/potusvn/.conda/envs/kickstarter/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [24]:
train = pd.read_csv('data/data_quantile_transform/train_quantile_transform.csv', encoding='latin1', low_memory=True)
test = pd.read_csv('data/data_quantile_transform/test_quantile_transform.csv', encoding='latin1', low_memory=True)
val = pd.read_csv('data/data_quantile_transform/val_quantile_transform.csv', encoding='latin1', low_memory=True)

In [46]:
train.shape

(185736, 222)

In [25]:
train_total = pd.concat([train, val], axis=0)

In [47]:
val.shape

(46434, 222)

In [26]:
train_total_x = train_total.drop(['success'], axis=1)
train_total_y = train_total.success

test_x = test.drop(['success'], axis=1)
test_y = test.success

In [ ]:
train_x.shape

# Step 1: Encode the data

## Train the network to encode data

In [14]:
input_dim = 221
encode_dim = 32
model = models.Sequential()
# Hidden - Layers
model.add(Dense(encode_dim, activation="relu", input_shape=(221,)))
# Output- Layer
model.add(Dense(input_dim, activation='sigmoid'))
model.summary()
# compiling the model
model.compile(
    optimizer='adam',
    loss="binary_crossentropy",
    metrics=["accuracy"]
)
# Define some callbacks
callbacks_func = []
earlyStopping = EarlyStopping(monitor='val_acc', patience=50, verbose=0, mode='max')
mcp_save = ModelCheckpoint('encode.hdf5', save_best_only=True, monitor='val_acc', mode='max')
callbacks_func = [mcp_save, earlyStopping]
          
results = model.fit(
    train_total_x, train_total_x,
    epochs=1000,
    batch_size=512,
    callbacks = callbacks_func,
    validation_data=(test_x, test_x)
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 32)                7104      
_________________________________________________________________
dense_8 (Dense)              (None, 221)               7293      
Total params: 14,397
Trainable params: 14,397
Non-trainable params: 0
_________________________________________________________________
Train on 232170 samples, validate on 99502 samples
Epoch 1/1000
232170/232170 [==============================] - 9s 37us/step - loss: -0.1672 - acc: 0.9155 - val_loss: -0.3097 - val_acc: 0.9341
Epoch 2/1000
232170/232170 [==============================] - 9s 37us/step - loss: -0.3224 - acc: 0.9365 - val_loss: -0.3322 - val_acc: 0.9394
Epoch 3/1000
232170/232170 [==============================] - 9s 37us/step - loss: -0.3405 - acc: 0.9408 - val_loss: -0.3449 - val_acc: 0.9418
Epoch 4/1000
232170/232170 [==========================

232170/232170 [==============================] - 9s 37us/step - loss: -0.3675 - acc: 0.9460 - val_loss: -0.3665 - val_acc: 0.9460
Epoch 54/1000
232170/232170 [==============================] - 9s 38us/step - loss: -0.3675 - acc: 0.9460 - val_loss: -0.3665 - val_acc: 0.9460
Epoch 55/1000
232170/232170 [==============================] - 9s 37us/step - loss: -0.3675 - acc: 0.9460 - val_loss: -0.3665 - val_acc: 0.9460
Epoch 56/1000
232170/232170 [==============================] - 9s 37us/step - loss: -0.3675 - acc: 0.9460 - val_loss: -0.3665 - val_acc: 0.9460
Epoch 57/1000
232170/232170 [==============================] - 9s 37us/step - loss: -0.3675 - acc: 0.9460 - val_loss: -0.3665 - val_acc: 0.9460
Epoch 58/1000
232170/232170 [==============================] - 9s 38us/step - loss: -0.3675 - acc: 0.9460 - val_loss: -0.3665 - val_acc: 0.9460
Epoch 59/1000
232170/232170 [==============================] - 9s 37us/step - loss: -0.3675 - acc: 0.9460 - val_loss: -0.3665 - val_acc: 0.9460
Epoch 

KeyboardInterrupt: 

In [19]:
encoder = models.Sequential([
    Dense(32, input_dim=221, activation="relu")]) # first number is output_dim

#set weights of the first layer
encoder.set_weights(model.layers[0].get_weights())

#compile it after setting the weights
encoder.compile(optimizer='adam', loss='binary_crossentropy')

In [59]:
train_total_encode_x = encoder.predict(train_total_x)

In [56]:
test_encode_x = encoder.predict(test_x)

In [60]:
train_total_x_new = pd.np.column_stack([train_total_x, train_total_encode_x])

In [62]:
test_x_new = pd.np.column_stack([test_x, test_encode_x])

## Train and test with new encoder data

In [67]:
train_x = train_total_x_new[:int(len(train_total_x_new)*0.8), :]
val_x = train_total_x_new[int(len(train_total_x_new)*0.8):, :]

In [73]:
train_y = train_total_y[:int(len(train_total_x_new)*0.8)]
val_y = train_total_y[int(len(train_total_x_new)*0.8):]

In [77]:
def build_model(hidden_layers, optimizer=optimizers.Adam(lr=0.01), batch_size=512, drop_out=0.0, l2_val=0, file_name=None):
    model = models.Sequential()
    # Hidden - Layers
    for idx, layer in enumerate(hidden_layers):
        if idx == 0:
            model.add(Dense(layer, activation="relu", input_shape=(253,), kernel_regularizer=l2(l2_val)))
            model.add(Dropout(drop_out, noise_shape=None, seed=None))
        else:
            model.add(Dense(layer, activation="relu", kernel_regularizer=l2(l2_val)))
            model.add(Dropout(drop_out, noise_shape=None, seed=None))
    # Output- Layer
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    # compiling the model
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    # Define some callbacks
    callbacks_func = []
    earlyStopping = EarlyStopping(monitor='val_acc', patience=50, verbose=0, mode='max')
    if file_name is None:
        callbacks_func = [earlyStopping]
    else:
        mcp_save = ModelCheckpoint(file_name+'.hdf5', save_best_only=True, monitor='val_acc', mode='max')
        callbacks_func = [mcp_save, earlyStopping]
    results = model.fit(
        train_x, train_y,
        epochs=1000,
        batch_size=batch_size,
        callbacks = callbacks_func,
        validation_data=(val_x, val_y)
    )
    return model, results

In [78]:
model_encode, result_encode = build_model(hidden_layers=[20,15,10], drop_out=0.1, l2_val=0.0001, optimizer=optimizers.Adam(lr=0.001),file_name='model_encode')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 20)                5080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 15)                315       
_________________________________________________________________
dropout_2 (Dropout)          (None, 15)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                160       
_________________________________________________________________
dropout_3 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 11        
Total para

185736/185736 [==============================] - 2s 11us/step - loss: 0.4310 - acc: 0.7875 - val_loss: 0.4287 - val_acc: 0.7865
Epoch 50/1000
185736/185736 [==============================] - 2s 10us/step - loss: 0.4321 - acc: 0.7870 - val_loss: 0.4296 - val_acc: 0.7838
Epoch 51/1000
185736/185736 [==============================] - 2s 10us/step - loss: 0.4305 - acc: 0.7880 - val_loss: 0.4274 - val_acc: 0.7868
Epoch 52/1000
185736/185736 [==============================] - 2s 11us/step - loss: 0.4310 - acc: 0.7880 - val_loss: 0.4292 - val_acc: 0.7849
Epoch 53/1000
185736/185736 [==============================] - 2s 10us/step - loss: 0.4304 - acc: 0.7882 - val_loss: 0.4264 - val_acc: 0.7877
Epoch 54/1000
185736/185736 [==============================] - 2s 11us/step - loss: 0.4316 - acc: 0.7874 - val_loss: 0.4271 - val_acc: 0.7872
Epoch 55/1000
185736/185736 [==============================] - 2s 11us/step - loss: 0.4309 - acc: 0.7874 - val_loss: 0.4305 - val_acc: 0.7872
Epoch 56/1000
185736

Epoch 107/1000
185736/185736 [==============================] - 2s 11us/step - loss: 0.4292 - acc: 0.7881 - val_loss: 0.4304 - val_acc: 0.7847
Epoch 108/1000
185736/185736 [==============================] - 2s 11us/step - loss: 0.4290 - acc: 0.7884 - val_loss: 0.4276 - val_acc: 0.7873
Epoch 109/1000
185736/185736 [==============================] - 2s 11us/step - loss: 0.4292 - acc: 0.7879 - val_loss: 0.4306 - val_acc: 0.7832
Epoch 110/1000
185736/185736 [==============================] - 2s 11us/step - loss: 0.4300 - acc: 0.7876 - val_loss: 0.4274 - val_acc: 0.7875
Epoch 111/1000
185736/185736 [==============================] - 2s 10us/step - loss: 0.4291 - acc: 0.7884 - val_loss: 0.4253 - val_acc: 0.7886
Epoch 112/1000
185736/185736 [==============================] - 2s 11us/step - loss: 0.4295 - acc: 0.7885 - val_loss: 0.4281 - val_acc: 0.7886
Epoch 113/1000
185736/185736 [==============================] - 2s 11us/step - loss: 0.4291 - acc: 0.7883 - val_loss: 0.4265 - val_acc: 0.7888

185736/185736 [==============================] - 2s 11us/step - loss: 0.4288 - acc: 0.7881 - val_loss: 0.4244 - val_acc: 0.7893
Epoch 165/1000
185736/185736 [==============================] - 2s 11us/step - loss: 0.4291 - acc: 0.7884 - val_loss: 0.4253 - val_acc: 0.7880
Epoch 166/1000
185736/185736 [==============================] - 2s 11us/step - loss: 0.4285 - acc: 0.7889 - val_loss: 0.4310 - val_acc: 0.7864
Epoch 167/1000
185736/185736 [==============================] - 2s 11us/step - loss: 0.4295 - acc: 0.7888 - val_loss: 0.4252 - val_acc: 0.7876
Epoch 168/1000
185736/185736 [==============================] - 2s 11us/step - loss: 0.4301 - acc: 0.7880 - val_loss: 0.4266 - val_acc: 0.7884
Epoch 169/1000
185736/185736 [==============================] - 2s 11us/step - loss: 0.4288 - acc: 0.7873 - val_loss: 0.4278 - val_acc: 0.7858
Epoch 170/1000
185736/185736 [==============================] - 2s 11us/step - loss: 0.4291 - acc: 0.7880 - val_loss: 0.4278 - val_acc: 0.7875
Epoch 171/1000

In [80]:
def build_model_without_train(hidden_layers, drop_out=0.0, l2_val=0, optimizer=optimizers.Adam(lr=0.000001)):
    model = models.Sequential()
    # Hidden - Layers
    for idx, layer in enumerate(hidden_layers):
        if idx == 0:
            model.add(Dense(layer, activation="relu", input_shape=(253,), kernel_regularizer=l2(l2_val)))
            model.add(Dropout(drop_out, noise_shape=None, seed=None))
        else:
            model.add(Dense(layer, activation="relu", kernel_regularizer=l2(l2_val)))
            model.add(Dropout(drop_out, noise_shape=None, seed=None))
    # Output- Layer
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    
    return model

In [81]:
final_model = build_model_without_train(hidden_layers=[20,15,10], drop_out=0.1, l2_val=0.0001)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 20)                5080      
_________________________________________________________________
dropout_4 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 15)                315       
_________________________________________________________________
dropout_5 (Dropout)          (None, 15)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                160       
_________________________________________________________________
dropout_6 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 11        
Total para

In [82]:
final_model.load_weights('model_encode.hdf5')

In [83]:
final_model.evaluate(test_x_new, test_y)

99502/99502 [==============================] - 4s 42us/step


[0.4230642681863329, 0.790335872646974]